In [1]:
import pandas as pd
ticks = pd.read_csv ('../data/manual/ticks.csv', sep="|")
ticks['gvkey'] = ticks['gvkey'].fillna(0.0).astype(int)
ticks['Permco'] = ticks['Permco'].fillna(0.0).astype(int)
ticks

,Primary Dealer,Holding Company,Ticker,Start Date,End Date,Permco,gvkey
0,ABN AMRO INCORPORATED,NaN,ABN,9/29/1998,9/15/2006,31989,15504
1,"AUBREY G. LANSTON & CO., INC.",Citigroup Inc.,C (NYSE),5/19/1960,4/17/2000,20483,3243
2,BA Securities,Bank of America Corporation,BAC (NYSE),4/18/1994,9/30/1997,437,7647
3,Banc One,JPMorgan Chase & Co. (Acquired by JPMorgan Chase),JPM (NYSE),4/1/1999,8/1/2004,20436,2968
4,Bank of America,Bank of America Corporation,BAC (NYSE),11/17/1971,4/15/1994,437,7647
...,...,...,...,...,...,...,...
106,"WERTHEIM SCHRODER & CO., INC.",Schroder Wertheim & Co. Inc.,NaN,6/24/1988,11/8/1990,0,0
107,WESTPAC POLLOCK GOV'T SECURITIES INC,Westpac Banking Corporation,WBK (NYSE),2/4/1987,6/27/1990,22027,15362
108,"WHITE, WELD & CO INC.","Merrill Lynch & Co., Inc.",NaN,2/26/1976,4/18/1978,21190,7267
109,"YAMAICHI INT'L (AMERICA), INC.",Yamaichi Securities Company Limited,NaN,9/29/1988,12/4/1997,0,0


In [2]:
import pandas as pd
import wrds
import config
from datetime import datetime

db = wrds.Connection(wrds_username=config.WRDS_USERNAME)

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [3]:
def fetch_financial_data(pgvkey, start_date, end_date):
    """
    Fetch financial data for a given ticker and date range from the CCM database in WRDS.
    
    :param gvkey: The gvkey symbol for the company.
    :param start_date: The start date for the data in YYYY-MM-DD format.
    :param end_date: The end date for the data in YYYY-MM-DD format or 'Current'.
    :return: A DataFrame containing the financial data.
    """
    if not gvkey:  # Skip if no ticker is available
        return pd.DataFrame()
    
    # If the end date is 'Current', replace it with today's date
    if end_date == 'Current':
        end_date = datetime.today().strftime('%Y-%m-%d')
    
    # Convert dates to the correct format if necessary
    start_date = pd.to_datetime(start_date).strftime('%Y-%m-%d')
    end_date = pd.to_datetime(end_date).strftime('%Y-%m-%d')
    
        
    query = f"""
    SELECT datadate, at AS total_assets, lt AS book_debt, ceq AS book_equity, csho*prcc_f AS market_equity, gvkey, conm
    FROM comp.funda as cst
    WHERE cst.gvkey = '{str(pgvkey).zfill(6)}'
    AND cst.datadate BETWEEN '{start_date}' AND '{end_date}'
    AND indfmt='INDL'
    AND datafmt='STD'
    AND popsrc='D'
    AND consol='C'
    """
    data = db.raw_sql(query)
    return data
    

Get all data for primary dealers

In [4]:
empty_tickers = []
prim_dealers = pd.DataFrame()
# Iterate over DataFrame rows and fetch data for each ticker
for index, row in ticks.iterrows():
    gvkey = row['gvkey']
    start_date = row['Start Date']
    end_date = row['End Date']      # Formatting date for the query
    
    # Fetch financial data for the ticker if available
    new_data = fetch_financial_data(gvkey, start_date, end_date)
    if isinstance(new_data, tuple):
        empty_tickers.append({row['Ticker']:gvkey})
    else:
        prim_dealers = pd.concat([new_data, prim_dealers], axis=0)
print(empty_tickers)

/var/folders/qg/z7bl5k3s0ps22f89lw_lrth80000gn/T/ipykernel_14562/3698523706.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  prim_dealers = pd.concat([new_data, prim_dealers], axis=0)
/var/folders/qg/z7bl5k3s0ps22f89lw_lrth80000gn/T/ipykernel_14562/3698523706.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  prim_dealers = pd.concat([new_data, prim_dealers], axis=0)
/var/folders/qg/z7bl5k3s0ps22f89lw_lrth80000gn/T/ipykernel_14562/3698523706.py:14: FutureWarning: The behavior of DataFrame 

[]


Get data from all firms in Compustat

In [5]:
prim_dealers

,datadate,total_assets,book_debt,book_equity,market_equity,gvkey,conm
0,1993-12-31,50782.297,47213.898,3226.500,7121.765625,008007,WELLS FARGO & CO
1,1994-12-31,59315.898,55469.500,3334.400,7226.264375,008007,WELLS FARGO & CO
2,1995-12-31,72134.375,66822.250,5009.797,11641.080000,008007,WELLS FARGO & CO
3,1996-12-31,80175.375,74111.188,5875.398,16038.580500,008007,WELLS FARGO & CO
4,1997-12-31,88540.188,81518.000,6834.199,29396.486250,008007,WELLS FARGO & CO
...,...,...,...,...,...,...,...
3,2001-12-31,531713.000,517166.000,9767.000,24997.940000,015504,ABN-AMRO HOLDINGS NV
4,2002-12-31,582984.873,567686.209,10450.400,25655.008000,015504,ABN-AMRO HOLDINGS NV
5,2003-12-31,705982.489,684869.917,15411.170,38457.492840,015504,ABN-AMRO HOLDINGS NV
6,2004-12-31,823953.817,797891.813,19230.729,44400.773200,015504,ABN-AMRO HOLDINGS NV


In [6]:
prim_dealers = prim_dealers.drop_duplicates()
prim_dealers

,datadate,total_assets,book_debt,book_equity,market_equity,gvkey,conm
0,1993-12-31,50782.297,47213.898,3226.500,7121.765625,008007,WELLS FARGO & CO
1,1994-12-31,59315.898,55469.500,3334.400,7226.264375,008007,WELLS FARGO & CO
2,1995-12-31,72134.375,66822.250,5009.797,11641.080000,008007,WELLS FARGO & CO
3,1996-12-31,80175.375,74111.188,5875.398,16038.580500,008007,WELLS FARGO & CO
4,1997-12-31,88540.188,81518.000,6834.199,29396.486250,008007,WELLS FARGO & CO
...,...,...,...,...,...,...,...
3,2001-12-31,531713.000,517166.000,9767.000,24997.940000,015504,ABN-AMRO HOLDINGS NV
4,2002-12-31,582984.873,567686.209,10450.400,25655.008000,015504,ABN-AMRO HOLDINGS NV
5,2003-12-31,705982.489,684869.917,15411.170,38457.492840,015504,ABN-AMRO HOLDINGS NV
6,2004-12-31,823953.817,797891.813,19230.729,44400.773200,015504,ABN-AMRO HOLDINGS NV


In [ ]:
import pandas as pd

def fill_monthly_within_report_periods(df):
    """
    对于每个 gvkey，将数据按月填充，但只填充在该 gvkey 报告期内（即首次和最后一次 datadate 之间）。
    
    :param df: DataFrame，包含 datadate（报告期）、gvkey、财务数据列
    :return: 变为月度数据的 DataFrame，前向填充，仅限于该 gvkey 有数据的报告期间
    """
    df = df.copy()
    df['datadate'] = pd.to_datetime(df['datadate'])

    # 存储填充后的数据
    filled_df_list = []

    # 按 gvkey 分组，分别填充每个公司的数据
    for gvkey, group in df.groupby('gvkey'):
        # 计算该 gvkey 的数据范围（从首次报告期到最后一次报告期）
        date_range = pd.date_range(start=group['datadate'].min(), 
                                   end=group['datadate'].max(), 
                                   freq='M')

        # 生成完整的月度 DataFrame
        complete_dates = pd.DataFrame({'datadate': date_range})
        complete_dates['gvkey'] = gvkey

        # 合并原始数据，并后向填充
        # merged = pd.merge(complete_dates, group, on=['datadate', 'gvkey'], how='left').ffill()
        merged = pd.merge(complete_dates, group, on=['datadate', 'gvkey'], how='left').bfill()


        # 只保留在该 gvkey 数据报告区间内的数据
        merged = merged[merged['datadate'].between(group['datadate'].min(), group['datadate'].max())]

        # 存入列表
        filled_df_list.append(merged)

    # 合并所有 gvkey 处理后的数据
    filled_df = pd.concat(filled_df_list, ignore_index=True)

    return filled_df


In [8]:
# yearly_sum_pd = prim_dealers.groupby('datadate').agg({
#     'total_assets': 'sum',
#     'book_debt': 'sum',
#     'book_equity': 'sum',
#     'market_equity': 'sum'
# }).reset_index()
### katarina
yearly_sum_pd = fill_monthly_within_report_periods(prim_dealers).reset_index(drop=True)
yearly_sum_pd

/var/folders/qg/z7bl5k3s0ps22f89lw_lrth80000gn/T/ipykernel_14562/2468786844.py:19: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  date_range = pd.date_range(start=group['datadate'].min(),
/var/folders/qg/z7bl5k3s0ps22f89lw_lrth80000gn/T/ipykernel_14562/2468786844.py:19: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  date_range = pd.date_range(start=group['datadate'].min(),
/var/folders/qg/z7bl5k3s0ps22f89lw_lrth80000gn/T/ipykernel_14562/2468786844.py:19: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  date_range = pd.date_range(start=group['datadate'].min(),
/var/folders/qg/z7bl5k3s0ps22f89lw_lrth80000gn/T/ipykernel_14562/2468786844.py:19: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  date_range = pd.date_range(start=group['datadate'].min(),
/var/folders/qg/z7bl5k3s0ps22f89

,datadate,gvkey,total_assets,book_debt,book_equity,market_equity,conm
0,1960-12-31,001414,891.200,386.297,NaN,NaN,PRIMERICA INC
1,1961-01-31,001414,891.200,386.297,NaN,NaN,PRIMERICA INC
2,1961-02-28,001414,891.200,386.297,NaN,NaN,PRIMERICA INC
3,1961-03-31,001414,891.200,386.297,NaN,NaN,PRIMERICA INC
4,1961-04-30,001414,891.200,386.297,NaN,NaN,PRIMERICA INC
...,...,...,...,...,...,...,...
10256,2023-11-30,248136,1912614.108,1843345.011,23538.0,35732.53866,MIZUHO FINANCIAL GROUP INC
10257,2023-12-31,248136,1912614.108,1843345.011,23538.0,35732.53866,MIZUHO FINANCIAL GROUP INC
10258,2024-01-31,248136,1912614.108,1843345.011,23538.0,35732.53866,MIZUHO FINANCIAL GROUP INC
10259,2024-02-29,248136,1912614.108,1843345.011,23538.0,35732.53866,MIZUHO FINANCIAL GROUP INC


In [9]:
def pull_CRSP_Comp_Link_Table(wrds_username=config.WRDS_USERNAME):
    sql_query = """
        SELECT 
            gvkey, lpermco AS permco, linktype, linkprim, linkdt, linkenddt, tic
        FROM 
            crsp.ccmxpf_linkhist
        WHERE 
            substr(linktype,1,1)='L' AND 
            (linkprim ='C' OR linkprim='P')
        """
    db = wrds.Connection(wrds_username=wrds_username)
    ccm = db.raw_sql(sql_query, date_cols=["linkdt", "linkenddt"])
    db.close()
    return ccm

In [10]:
# pull_CRSP_Comp_Link_Table()

In [11]:
linktable = pd.read_csv('../data/manual/linktable.csv')
linktable

,gvkey,conm,tic,sic,LINKPRIM,LIID,LINKTYPE,LPERMNO,LPERMCO,LINKDT,LINKENDDT
0,1000,A & E PLASTIK PAK INC,AE.2,3089.0,P,01,LU,25881,23369,11/13/1970,6/30/1978
1,1001,A & M FOOD SERVICES INC,AMFD.,5812.0,P,01,LU,10015,6398,9/20/1983,7/31/1986
2,1002,AAI CORP,AAIC.1,3825.0,C,01,LC,10023,22159,12/14/1972,6/5/1973
3,1003,A.A. IMPORTING CO INC,ANTQ,5712.0,C,01,LU,10031,6672,12/7/1983,8/16/1989
4,1004,AAR CORP,AIR,5080.0,P,01,LU,54594,20000,4/24/1972,E
...,...,...,...,...,...,...,...,...,...,...,...
31944,349994,CLEARMIND MEDICINE INC,CMND,2834.0,P,01,LC,23514,59438,11/15/2022,E
31945,350681,GETNET ADQUIRENCIA E,GET,7374.0,P,90,LC,22205,58855,10/22/2021,3/31/2023
31946,351038,QUOIN PHARMACEUTICALS LTD,QNRX,2834.0,P,90,LC,16161,55612,10/29/2021,E
31947,352262,COOL COMPANY LTD,CLCO,4400.0,P,01,LC,23773,59507,3/17/2023,E


In [12]:
merged = pd.merge(ticks, linktable, left_on='gvkey', right_on='gvkey')
merged

,Primary Dealer,Holding Company,Ticker,Start Date,End Date,Permco,gvkey,conm,tic,sic,LINKPRIM,LIID,LINKTYPE,LPERMNO,LPERMCO,LINKDT,LINKENDDT
0,ABN AMRO INCORPORATED,NaN,ABN,9/29/1998,9/15/2006,31989,15504,ABN-AMRO HOLDINGS NV,ABNYY,6020.0,P,90,LC,84766,31989,5/21/1997,4/24/2008
1,"AUBREY G. LANSTON & CO., INC.",Citigroup Inc.,C (NYSE),5/19/1960,4/17/2000,20483,3243,CITIGROUP INC,C,6199.0,P,01,LC,70519,20483,10/29/1986,E
2,BA Securities,Bank of America Corporation,BAC (NYSE),4/18/1994,9/30/1997,437,7647,BANK OF AMERICA CORP,BAC,6020.0,P,01,LC,59408,3151,12/14/1972,E
3,Banc One,JPMorgan Chase & Co. (Acquired by JPMorgan Chase),JPM (NYSE),4/1/1999,8/1/2004,20436,2968,JPMORGAN CHASE & CO,JPM,6020.0,P,01,LC,47896,20436,3/5/1969,E
4,Bank of America,Bank of America Corporation,BAC (NYSE),11/17/1971,4/15/1994,437,7647,BANK OF AMERICA CORP,BAC,6020.0,P,01,LC,59408,3151,12/14/1972,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,"WEEDEN & CO., INC.","Merrill Lynch & Co., Inc.",NaN,6/17/1976,5/15/1978,21190,7267,MERRILL LYNCH & CO INC,BAC2,6211.0,P,01,LC,52919,21190,7/27/1971,12/31/2008
94,WESTPAC POLLOCK GOV'T SECURITIES INC,Westpac Banking Corporation,WBK (NYSE),2/4/1987,6/27/1990,22027,15362,WESTPAC BANKING CORP,WEBNF,6020.0,C,90,LC,75466,22027,1/1/2022,1/28/2022
95,WESTPAC POLLOCK GOV'T SECURITIES INC,Westpac Banking Corporation,WBK (NYSE),2/4/1987,6/27/1990,22027,15362,WESTPAC BANKING CORP,WEBNF,6020.0,P,90,LC,75466,22027,3/17/1989,12/31/2021
96,"WHITE, WELD & CO INC.","Merrill Lynch & Co., Inc.",NaN,2/26/1976,4/18/1978,21190,7267,MERRILL LYNCH & CO INC,BAC2,6211.0,P,01,LC,52919,21190,7/27/1971,12/31/2008


### Get all data from broker dealers with SIC code 6211 or 6221 that are not in list of primary dealers


In [13]:
linked_bd = linktable[((linktable['sic'] == 6211) | (linktable['sic'] == 6221)) & (~linktable['gvkey'].isin(ticks['gvkey'].tolist()))]
linked_bd

,gvkey,conm,tic,sic,LINKPRIM,LIID,LINKTYPE,LPERMNO,LPERMCO,LINKDT,LINKENDDT
167,1148,ADVEST GROUP INC,ADV.2,6211.0,P,01,LU,64514,420,11/21/1980,2/28/2001
987,1967,BACHE GROUP INC,BAC.,6211.0,P,01,LU,54893,25401,5/31/1972,6/11/1981
1379,2330,BOSTON CO INC -CL B,BSTNB,6211.0,C,00X,LU,18877,22881,1/1/1975,1/30/1975
1380,2330,BOSTON CO INC -CL B,BSTNB,6211.0,C,01,LU,18877,22881,1/31/1975,9/30/1981
1475,2432,BROWN (ROBERT C.) & CO INC,RCBI,6211.0,P,01,LU,19538,6441,9/30/1983,11/30/1989
...,...,...,...,...,...,...,...,...,...,...,...
30429,178073,B. RILEY FINANCIAL INC,RILY,6211.0,P,03,LC,15540,55369,7/16/2015,E
30479,178519,LPL FINANCIAL HOLDINGS INC,LPLA,6211.0,P,01,LC,12400,53576,11/18/2010,E
30548,178848,DIRECT MARKETS HOLDINGS CORP,MKTSQ,6211.0,P,01,LC,92312,52781,10/16/2007,9/13/2012
31100,185518,GLOBAL BROKERAGE INC,GLBR,6211.0,C,01,LC,12446,53607,12/2/2010,E


In [14]:
empty_tickers = []
broker_dealers = pd.DataFrame()
for index, row in linked_bd.iterrows():
    gvkey = row['gvkey']
    start_date = '1960-01-01'
    end_date = '2012-12-31'    
    new_data = fetch_financial_data(gvkey, start_date, end_date)
    if isinstance(new_data, tuple):
        empty_tickers.append({row['conm']:gvkey})
    else:
        broker_dealers = pd.concat([new_data, broker_dealers], axis=0)
print(empty_tickers)

/var/folders/qg/z7bl5k3s0ps22f89lw_lrth80000gn/T/ipykernel_14562/2850304366.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  broker_dealers = pd.concat([new_data, broker_dealers], axis=0)
/var/folders/qg/z7bl5k3s0ps22f89lw_lrth80000gn/T/ipykernel_14562/2850304366.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  broker_dealers = pd.concat([new_data, broker_dealers], axis=0)
/var/folders/qg/z7bl5k3s0ps22f89lw_lrth80000gn/T/ipykernel_14562/2850304366.py:11: FutureWarning: The behavior of Da

[]


In [15]:
broker_dealers = broker_dealers.drop_duplicates()
broker_dealers

,datadate,total_assets,book_debt,book_equity,market_equity,gvkey,conm
0,1999-12-31,842.252,729.293,NaN,NaN,209386,VAN DER MOOLEN NV
1,2000-12-31,876.311,726.460,NaN,NaN,209386,VAN DER MOOLEN NV
2,2001-12-31,3631.252,3064.614,538.155,1098.808600,209386,VAN DER MOOLEN NV
3,2002-12-31,1567.193,1174.844,327.342,825.624310,209386,VAN DER MOOLEN NV
4,2003-12-31,1114.079,818.679,253.830,333.861110,209386,VAN DER MOOLEN NV
...,...,...,...,...,...,...,...
15,1996-09-30,965.177,875.935,89.242,81.929250,001148,ADVEST GROUP INC
16,1997-09-30,1078.839,973.186,105.653,227.151465,001148,ADVEST GROUP INC
17,1998-09-30,1233.271,1109.604,123.667,182.335875,001148,ADVEST GROUP INC
18,1999-09-30,1462.621,1327.485,135.136,162.881250,001148,ADVEST GROUP INC


In [16]:
# yearly_sum_bd = broker_dealers.groupby('datadate').agg({
#     'total_assets': 'sum',
#     'book_debt': 'sum',
#     'book_equity': 'sum',
#     'market_equity': 'sum'
# }).reset_index()
###katarina
yearly_sum_bd = fill_monthly_within_report_periods(broker_dealers).reset_index(drop=True)
yearly_sum_bd = yearly_sum_bd.rename(columns={'total_assets':'total_assets_bd', 'book_debt':'book_debt_bd', 'book_equity':'book_equity_bd','market_equity':'market_equity_bd' })
yearly_sum_bd

/var/folders/qg/z7bl5k3s0ps22f89lw_lrth80000gn/T/ipykernel_14562/2468786844.py:19: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  date_range = pd.date_range(start=group['datadate'].min(),
/var/folders/qg/z7bl5k3s0ps22f89lw_lrth80000gn/T/ipykernel_14562/2468786844.py:19: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  date_range = pd.date_range(start=group['datadate'].min(),
/var/folders/qg/z7bl5k3s0ps22f89lw_lrth80000gn/T/ipykernel_14562/2468786844.py:19: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  date_range = pd.date_range(start=group['datadate'].min(),
/var/folders/qg/z7bl5k3s0ps22f89lw_lrth80000gn/T/ipykernel_14562/2468786844.py:19: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  date_range = pd.date_range(start=group['datadate'].min(),
/var/folders/qg/z7bl5k3s0ps22f89

,datadate,gvkey,total_assets_bd,book_debt_bd,book_equity_bd,market_equity_bd,conm
0,1981-09-30,001148,185.469,165.845,18.617,18.847482,ADVEST GROUP INC
1,1981-10-31,001148,185.469,165.845,18.617,18.847482,ADVEST GROUP INC
2,1981-11-30,001148,185.469,165.845,18.617,18.847482,ADVEST GROUP INC
3,1981-12-31,001148,185.469,165.845,18.617,18.847482,ADVEST GROUP INC
4,1982-01-31,001148,185.469,165.845,18.617,18.847482,ADVEST GROUP INC
...,...,...,...,...,...,...,...
18238,2007-08-31,209386,2199.940,1891.526,289.806,255.239340,VAN DER MOOLEN NV
18239,2007-09-30,209386,2199.940,1891.526,289.806,255.239340,VAN DER MOOLEN NV
18240,2007-10-31,209386,2199.940,1891.526,289.806,255.239340,VAN DER MOOLEN NV
18241,2007-11-30,209386,2199.940,1891.526,289.806,255.239340,VAN DER MOOLEN NV


In [17]:
merged_pd_bd = pd.merge(yearly_sum_pd, yearly_sum_bd, left_on='datadate', right_on='datadate')
merged_pd_bd.index = pd.to_datetime(merged_pd_bd['datadate'])
merged_pd_bd['month'] = pd.to_datetime(merged_pd_bd['datadate']).dt.month
merged_pd_bd = merged_pd_bd.groupby('month').agg({
    'total_assets': 'sum',
    'book_debt': 'sum',
    'book_equity': 'sum',
    'market_equity': 'sum',
    'total_assets_bd': 'sum',
    'book_debt_bd': 'sum',
    'book_equity_bd': 'sum',
    'market_equity_bd': 'sum'
})
merged_pd_bd

,total_assets,book_debt,book_equity,market_equity,total_assets_bd,book_debt_bd,book_equity_bd,market_equity_bd
month,,,,,,,,
1,9.062849e+09,8.584724e+09,4.276186e+08,7.014034e+08,1.982442e+08,1.872892e+08,1.002281e+07,1.294258e+07
2,9.034449e+09,8.557820e+09,4.262174e+08,6.982152e+08,1.983761e+08,1.873849e+08,1.005902e+07,1.302533e+07
3,9.115136e+09,8.634669e+09,4.278218e+08,7.002199e+08,2.000184e+08,1.889507e+08,1.012794e+07,1.310089e+07
4,9.037629e+09,8.561618e+09,4.238222e+08,6.944057e+08,2.001982e+08,1.891565e+08,1.010413e+07,1.305580e+07
5,9.034531e+09,8.558669e+09,4.236849e+08,6.941524e+08,2.002049e+08,1.891580e+08,1.010939e+07,1.306368e+07
6,9.032036e+09,8.556252e+09,4.235998e+08,6.935456e+08,2.020573e+08,1.909485e+08,1.016304e+07,1.312726e+07
7,8.982110e+09,8.509333e+09,4.209393e+08,6.914045e+08,2.018273e+08,1.907304e+08,1.015281e+07,1.312203e+07
8,8.976666e+09,8.504149e+09,4.206949e+08,6.905703e+08,2.017996e+08,1.907051e+08,1.015049e+07,1.311890e+07
9,8.997820e+09,8.524106e+09,4.218138e+08,6.923138e+08,2.030334e+08,1.917587e+08,1.032564e+07,1.345603e+07


In [18]:
merged_pd_bd['tot_assets_rat'] = merged_pd_bd['total_assets']/(merged_pd_bd['total_assets_bd']+merged_pd_bd['total_assets'])
merged_pd_bd['book_debt_rat'] = merged_pd_bd['book_debt']/(merged_pd_bd['book_debt_bd']+merged_pd_bd['book_debt'])
merged_pd_bd['book_equity_rat'] = merged_pd_bd['book_equity']/(merged_pd_bd['book_equity_bd']+merged_pd_bd['book_equity'])
merged_pd_bd['market_equity_rat'] = merged_pd_bd['market_equity']/(merged_pd_bd['market_equity_bd']+merged_pd_bd['market_equity'])
merged_pd_bd[['tot_assets_rat', 'book_debt_rat', 'book_equity_rat','market_equity_rat']].mean()

tot_assets_rat       0.978169
book_debt_rat        0.978224
book_equity_rat      0.976590
market_equity_rat    0.981430
dtype: float64

Get data from banks
6011 - Federal Reserve Banks
6021 - National Commercial Banks
6022 - State Commercial Banks
6029 - Commercial Banks, nec (not elsewhere classified)
6081 - Foreign Bank and Branches and Agencies
6082 - Foreign Trade and International Banking Institutions

In [19]:
bank_sic = [6011,6021,6022,6029,6081,6082,6020] # these are not all the codes, in the process of looking these up
linked_banks = linktable[(linktable['sic'].isin(bank_sic)) & (~linktable['gvkey'].isin(ticks['gvkey'].tolist()))]
linked_banks.drop_duplicates()

,gvkey,conm,tic,sic,LINKPRIM,LIID,LINKTYPE,LPERMNO,LPERMCO,LINKDT,LINKENDDT
197,1178,AFFILIATED BANKSHARES COLO,AFBK.,6020.0,C,00X,LU,11173,94,1/1/1974,1/30/1974
198,1178,AFFILIATED BANKSHARES COLO,AFBK.,6020.0,P,01,LU,11173,94,1/31/1974,11/30/1992
309,1290,ALLIED BANCSHARES INC-TX,ALBN,6020.0,P,01,LC,11870,151,1/31/1974,1/29/1988
310,1290,ALLIED BANCSHARES INC-TX,ALBN,6020.0,C,00X,LC,11870,151,1/1/1974,1/30/1974
485,1458,AMERICAN FLETCHER CORP,AFLT,6020.0,C,01,LU,12969,106,12/14/1972,1/23/1987
...,...,...,...,...,...,...,...,...,...,...,...
31673,260758,ATLANTIC SOUTHERN FINL GRP,ASFN,6020.0,C,01,LC,91955,52130,4/16/2007,1/28/2011
31703,264418,1ST CENTURY BANCSHARES INC,FCTY,6020.0,P,01,LC,92870,53112,12/18/2008,6/30/2016
31721,266160,CITIZENS COMMUNITY BANCORP,CZWI,6020.0,P,01,LC,91609,51445,11/1/2006,E
31807,285313,BANCO SANTANDER BRASIL -ADR,BSBR,6020.0,P,90,LC,93054,53230,10/7/2009,E


In [20]:
def get_comparison_group_data(linktable_df, start_date, end_date):
    empty_tickers = []
    comparison_group_df = pd.DataFrame()
    for index, row in linktable_df.iterrows():
        gvkey = row['gvkey']   
        new_data = fetch_financial_data(gvkey, start_date, end_date)
        if isinstance(new_data, tuple):
            empty_tickers.append({row['conm']:gvkey})
        else:
            comparison_group_df = pd.concat([new_data, comparison_group_df], axis=0)
    print('Empty Tickers: %s' % empty_tickers)
    return comparison_group_df

In [21]:
banks = get_comparison_group_data(linked_banks, '1960-01-01', '2012-12-31')
banks = banks.drop_duplicates()

/var/folders/qg/z7bl5k3s0ps22f89lw_lrth80000gn/T/ipykernel_14562/2803365408.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  comparison_group_df = pd.concat([new_data, comparison_group_df], axis=0)
/var/folders/qg/z7bl5k3s0ps22f89lw_lrth80000gn/T/ipykernel_14562/2803365408.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  comparison_group_df = pd.concat([new_data, comparison_group_df], axis=0)
/var/folders/qg/z7bl5k3s0ps22f89lw_lrth80000gn/T/ipykernel_14562/2803365408.py:10: FutureWarning

Empty Tickers: []


/var/folders/qg/z7bl5k3s0ps22f89lw_lrth80000gn/T/ipykernel_14562/2803365408.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  comparison_group_df = pd.concat([new_data, comparison_group_df], axis=0)


In [22]:
banks

,datadate,total_assets,book_debt,book_equity,market_equity,gvkey,conm
0,2007-12-31,60887.708,55149.327,5738.349,NaN,285313,BANCO SANTANDER BRASIL -ADR
1,2008-12-31,126915.378,105415.604,21497.497,NaN,285313,BANCO SANTANDER BRASIL -ADR
2,2009-12-31,181332.899,141581.540,39750.591,52929.469060,285313,BANCO SANTANDER BRASIL -ADR
3,2010-12-31,225279.708,181167.271,44107.581,51638.506400,285313,BANCO SANTANDER BRASIL -ADR
4,2011-12-31,214680.884,172789.011,41881.694,30876.314260,285313,BANCO SANTANDER BRASIL -ADR
...,...,...,...,...,...,...,...
13,1987-12-31,2536.776,2354.405,171.473,121.224000,001178,AFFILIATED BANKSHARES COLO
14,1988-12-31,2616.952,2439.173,167.005,97.241375,001178,AFFILIATED BANKSHARES COLO
15,1989-12-31,2611.094,2425.620,174.703,122.208000,001178,AFFILIATED BANKSHARES COLO
16,1990-12-31,2738.097,2545.413,181.912,109.897250,001178,AFFILIATED BANKSHARES COLO


In [23]:
linked_banks['sic'].value_counts()

sic
6020.0    1412
6022.0       1
Name: count, dtype: int64

In [24]:
# yearly_sum_banks = banks.groupby('datadate').agg({
#     'total_assets': 'sum',
#     'book_debt': 'sum',
#     'book_equity': 'sum',
#     'market_equity': 'sum'
# }).reset_index()
###katarina
yearly_sum_banks = fill_monthly_within_report_periods(banks).reset_index(drop=True)
yearly_sum_banks = yearly_sum_banks.rename(columns={'total_assets':'total_assets_bd', 'book_debt':'book_debt_bd', 'book_equity':'book_equity_bd','market_equity':'market_equity_bd' })
yearly_sum_banks

/var/folders/qg/z7bl5k3s0ps22f89lw_lrth80000gn/T/ipykernel_14562/2468786844.py:19: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  date_range = pd.date_range(start=group['datadate'].min(),
/var/folders/qg/z7bl5k3s0ps22f89lw_lrth80000gn/T/ipykernel_14562/2468786844.py:19: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  date_range = pd.date_range(start=group['datadate'].min(),
/var/folders/qg/z7bl5k3s0ps22f89lw_lrth80000gn/T/ipykernel_14562/2468786844.py:19: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  date_range = pd.date_range(start=group['datadate'].min(),
/var/folders/qg/z7bl5k3s0ps22f89lw_lrth80000gn/T/ipykernel_14562/2468786844.py:19: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  date_range = pd.date_range(start=group['datadate'].min(),
/var/folders/qg/z7bl5k3s0ps22f89

,datadate,gvkey,total_assets_bd,book_debt_bd,book_equity_bd,market_equity_bd,conm
0,1974-12-31,001178,810.549,754.215,48.549,26.380498,AFFILIATED BANKSHARES COLO
1,1975-01-31,001178,810.549,754.215,48.549,26.380498,AFFILIATED BANKSHARES COLO
2,1975-02-28,001178,810.549,754.215,48.549,26.380498,AFFILIATED BANKSHARES COLO
3,1975-03-31,001178,810.549,754.215,48.549,26.380498,AFFILIATED BANKSHARES COLO
4,1975-04-30,001178,810.549,754.215,48.549,26.380498,AFFILIATED BANKSHARES COLO
...,...,...,...,...,...,...,...
191130,2012-08-31,285313,214680.884,172789.011,41881.694,30876.314260,BANCO SANTANDER BRASIL -ADR
191131,2012-09-30,285313,214680.884,172789.011,41881.694,30876.314260,BANCO SANTANDER BRASIL -ADR
191132,2012-10-31,285313,214680.884,172789.011,41881.694,30876.314260,BANCO SANTANDER BRASIL -ADR
191133,2012-11-30,285313,214680.884,172789.011,41881.694,30876.314260,BANCO SANTANDER BRASIL -ADR


In [25]:
merged_pd_banks = pd.merge(yearly_sum_pd, yearly_sum_banks, left_on='datadate', right_on='datadate')
merged_pd_banks.index = pd.to_datetime(merged_pd_banks['datadate'])
merged_pd_banks['month'] = pd.to_datetime(merged_pd_banks['datadate']).dt.month
merged_pd_banks = merged_pd_banks.groupby('month').agg({
    'total_assets': 'sum',
    'book_debt': 'sum',
    'book_equity': 'sum',
    'market_equity': 'sum',
    'total_assets_bd': 'sum',
    'book_debt_bd': 'sum',
    'book_equity_bd': 'sum',
    'market_equity_bd': 'sum'
})
merged_pd_banks

,total_assets,book_debt,book_equity,market_equity,total_assets_bd,book_debt_bd,book_equity_bd,market_equity_bd
month,,,,,,,,
1,1.075666e+11,1.018773e+11,5.086453e+09,8.234575e+09,2.683418e+09,2.495086e+09,1.707658e+08,2.720915e+08
2,1.075666e+11,1.018773e+11,5.086453e+09,8.234575e+09,2.683418e+09,2.495086e+09,1.707658e+08,2.720915e+08
3,1.089092e+11,1.031596e+11,5.116198e+09,8.274314e+09,2.800174e+09,2.605909e+09,1.759308e+08,2.783677e+08
4,1.086636e+11,1.029279e+11,5.104197e+09,8.263763e+09,2.696677e+09,2.507386e+09,1.716842e+08,2.738985e+08
5,1.086595e+11,1.029239e+11,5.104033e+09,8.263364e+09,2.696673e+09,2.507384e+09,1.716827e+08,2.738962e+08
6,1.087414e+11,1.030015e+11,5.108009e+09,8.270318e+09,2.697588e+09,2.508223e+09,1.717555e+08,2.739842e+08
7,1.085388e+11,1.028110e+11,5.097210e+09,8.259675e+09,2.695050e+09,2.505830e+09,1.716176e+08,2.738375e+08
8,1.085388e+11,1.028110e+11,5.097210e+09,8.259675e+09,2.695050e+09,2.505830e+09,1.716176e+08,2.738375e+08
9,1.086796e+11,1.029442e+11,5.104252e+09,8.270507e+09,2.720480e+09,2.529695e+09,1.730669e+08,2.761231e+08


In [26]:
merged_pd_banks['tot_assets_rat'] = merged_pd_banks['total_assets']/(merged_pd_banks['total_assets_bd']+merged_pd_banks['total_assets'])
merged_pd_banks['book_debt_rat'] = merged_pd_banks['book_debt']/(merged_pd_banks['book_debt_bd']+merged_pd_banks['book_debt'])
merged_pd_banks['book_equity_rat'] = merged_pd_banks['book_equity']/(merged_pd_banks['book_equity_bd']+merged_pd_banks['book_equity'])
merged_pd_banks['market_equity_rat'] = merged_pd_banks['market_equity']/(merged_pd_banks['market_equity_bd']+merged_pd_banks['market_equity'])
merged_pd_banks[['tot_assets_rat', 'book_debt_rat', 'book_equity_rat','market_equity_rat']].mean()
# these ratios are between 0.1 and 0.2 off of the paper's numbers for banks

tot_assets_rat       0.975735
book_debt_rat        0.976169
book_equity_rat      0.967455
market_equity_rat    0.967927
dtype: float64

In [27]:
tick_inv = ticks.copy()
tick_inv['End Date'] = pd.to_datetime(tick_inv['End Date'].replace({'Current': datetime.today()}))

# Now apply the filter
tick_inv_filtered = tick_inv[tick_inv['End Date'] <= '2012-12-31']
tick_inv_filtered = tick_inv_filtered[tick_inv_filtered['gvkey'] == 0]
tick_inv_filtered

,Primary Dealer,Holding Company,Ticker,Start Date,End Date,Permco,gvkey
12,Becker Paribas Inc.,Banque Paribas,BNP (Euronext Paris),11/17/1971,1984-09-10,0,0
19,"Brophy, Gestal, Knight",NaN,NaN,5/8/1987,1988-06-19,0,0
23,CARROLL MCENTEE & MCGINLEY INC.,NaN,NaN,9/29/1976,1994-05-06,0,0
24,"C.F. CHILDS & CO., INC",NaN,NaN,5/19/1960,1965-06-29,0,0
26,CHEMICAL SECURITIES INC,Chemical Bank Corporation,CHL (NYSE),5/19/1960,1996-03-31,0,0
42,Drexel Burnham Lambert,NaN,NaN,5/19/1960,1990-03-28,0,0
43,"D.W. RICH & CO., INC",NaN,NaN,5/19/1960,1969-12-31,0,0
44,EASTBRIDGE CAPITAL INC.,NaN,NaN,6/18/1992,1998-05-29,0,0
45,FI Dupont,NaN,NaN,12/12/1968,1973-07-18,0,0
50,FIRST PENNCO SEC. INC.,NaN,NaN,3/7/1974,1980-08-27,0,0


In [28]:
tick_inv_filtered[['Primary Dealer', 'Start Date']].values.tolist()

[['Becker Paribas Inc.', '11/17/1971'],
 ['Brophy, Gestal, Knight', '5/8/1987'],
 ['CARROLL MCENTEE & MCGINLEY INC.', '9/29/1976'],
 ['C.F. CHILDS & CO., INC', '5/19/1960'],
 ['CHEMICAL SECURITIES INC', '5/19/1960'],
 ['Drexel Burnham Lambert', '5/19/1960'],
 ['D.W. RICH & CO., INC', '5/19/1960'],
 ['EASTBRIDGE CAPITAL INC.', '6/18/1992'],
 ['FI Dupont', '12/12/1968'],
 ['FIRST PENNCO SEC. INC.', '3/7/1974'],
 ['HARRIS GOVERNMENT SECURITIES', '7/15/1965'],
 ['IRVING SECURITIES, INC.', '5/19/1960'],
 ["KLEINWORT BENSON  GOV'T SEC., INC.", '2/13/1980'],
 ['MALON S. ANDRUS INC.', '5/19/1960'],
 ['NationsBanc', '7/6/1993'],
 ['THE NIKKO SECURITIES CO. INT', '12/22/1987'],
 ["NUVEEN GOV'T SEC. INC.", '11/18/1971'],
 ['NY Hanseatic', '2/8/1984'],
 ['PARIBAS CORPORATION', '5/1/1997'],
 ['SANWA SECURITIES (USA) CO., L', '6/20/1988'],
 ['SBC WARBURG DILLON READ INC.', '3/29/1990'],
 ['SECOND DISTRICT SECURITIES CO., INC', '6/15/1961'],
 ['Securities Groups', '5/19/1960'],
 ['SG COWEN SECURITIES

Creation of table 2 writeup: Table 2 was the first table our group was tasked with replicating. This table involves getting numbers such as total assets or market equity from CRSP Compustat database for different groups of firms, with the limitation that the firms be US domestic and have data available in CRSP Compustat. These groups were comprised of the primary dealers of the New York Fed, which are detailed in table A.1, all broker dealers, all banks, and all firms in Compustat. There were three time ranges that we observed, 1960-2012, 1960-1990, and 1990-2012. Replicating table 2 turned out to be a lot more involved than initially expected, due largely to the condition that we use the data of holding companies of the primary dealers rather than of the actual primary dealer. This required us to manually go through each individual dealer in the list of 112, and look up information about their holding company during a specified time period where they were considered a primary dealer. This was not a simple task because there were many cases where holding companies shifted multiple time within the specified period - forcing us to try and guess how the original author of the paper made this decision. We ended up choosing the holding company that held the primary dealer for the longest period of time during the specified interval. An alternative method would be to have multiple holding company data pulls for a single primary dealer, which may be the most accurate and most time consuming methodology. What was also a challenge was that firms seemed to have multiple names that they went by, making research and lookups of these firms difficult and easy to make an error.

After finishing the main lookup file, ticks.csv, which including all primary dealers listed in table A.1, their holding companies, their start and end dates, and their permcos and gvkeys (identifiers for CRSP and Compustat respectively) we then focused on pulling data from the WRDS database. After establishing a connection with WRDS we were able to query data from multiple sources. We made use of the CRSP Compustat historical link table to do most of this research, as well as some holding companies lookups on Google and chatGPT. We queried a full list of link table data from the WRDS website and downloaded it in Excel format to be able to filter and make notes. We also queried the link table from the WRDS historical table and used this table to help merge SIC codes with our completed ticks.csv dataset. For the financials dataset, we opted to query data by gvkey since the data for table 2 was specifically for company level financials which are found through Compustat.
  
After having the primary dealer data in the proper format of total assets, total book debt, book equity, and market equity, it was now time to create our subgroups and query data for them. The first one we worked on was the broker dealer dataset, which, as specified in the paper, included all primary dealers in our main dataset plus any firms with the SIC code 6211 or 6221. Using the historical linktable and our primary dealer linktable we created our subset and then used it to query financial data from Compustat. Now with our two different subsets of data, we merged them on the date key and then performed our ratio calculation. This calculation was the total value of primary dealers divided by the sum of the total value of primary dealers and the total value of non-primary dealer broker dealers. We then repeated this methodology for the banks comparison group which was generally comprised of firms with the SIC code 6020. We found that the bank ratios were farther from the paper than the broker dealer ratios. This could be for a multitude of reasons, potentially from differences in how holding companies were handled. 